# Read MYD35_L2, MYD06_L2 and LID_L2_05km_MLay products and identify the relevant properties

In [1]:
from pyhdf.HDF import *
from pyhdf.SD import *
from pyhdf.V import *
from caliop import Caliop_hdf_reader

CALIOP_pre_path = "./test_data/CALIOP/2017/01/"
MODIS_pre_path = "./test_data/MODIS/2017/01/"




# Use collocation database to load CALIOP and MODIS products